# Items demand utility

Import all the needed modules

In [1]:
import pandas as pd
from pandas import ExcelWriter
import numpy as np

Import from the statistics file the columns needed

In [2]:
excel_file = 'R12.xlsx'
df = pd.read_excel(excel_file, usecols = ['Item', 'Year month',
                                          'Sold qty', 'Used qty',
                                          'Not rep usage'])
df.head()

,Item,Year month,Sold qty,Used qty,Not rep usage
0,7ACEL1432 Termostato RCL,1810,0,0.0,1.0
1,7ACEL1432 Termostato RCL,1902,0,0.0,0.0
2,7ACEL1432 Termostato RCL,1903,0,0.0,0.0
3,645W1025 Sect.Imp./No Cont/ II / +1 fan,1901,0,0.0,1.0
4,645W1025 Sect.Imp./No Cont/ II / +1 fan,1902,0,0.0,0.0


Import from the master file the columns needed

In [3]:
master_file = 'Item master.xlsx'
df_master = pd.read_excel(master_file, usecols = ['Item no', 'Item name',
                                            'Product group descr'])

Get the first word of the item column using str.split and str.get function

In [4]:
df['Item no'] = df['Item'].str.split().str.get(0)

Remove possible whitespaces in the figures

In [5]:
columns = ['Sold qty', 'Used qty', 'Not rep usage']
for column in columns:
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace(' ', '')
    df[column] = df[column].astype(float)

Get the demand

In [6]:
df['Demand'] = df['Sold qty'] + df['Used qty'] + np.where(df['Not rep usage'] > 0, df['Not rep usage'], 0)

Get the pivot table of the demand and year month and replace NaN with zeros

In [7]:
pivot_df = pd.pivot_table(df, values = 'Demand', index = 'Item no' , columns = 'Year month', aggfunc=np.sum)
pivot_df = pivot_df.fillna(0)
pivot_df = pd.DataFrame(pivot_df.to_records())

Get the 'Item name' for each item. Merge it with the master_df

In [8]:
demand_df = pd.merge(df_master, pivot_df, how='left', left_on='Item no', right_on='Item no')
demand_df = demand_df.fillna(0)
demand_df.head()

,Item no,Item name,Product group descr,1809,1810,1811,1812,1901,1902,1903,1904,1905,1906,1907,1908
0,7ACEL1432,Termostato RCL,701 Hydronic (Water born) systems,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7ACEL1445,Termostato RCL,701 Hydronic (Water born) systems,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7ACTLO391,Support feet for AWC,701 Hydronic (Water born) systems,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7ACVF0481,Filter HAN-15,702 DX (Refrigerant) systems,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7HL023002,Roof Top HAN-13,702 DX (Refrigerant) systems,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Output the demand to Excel

In [9]:
name = 'Items Demand.xlsx'
writer = pd.ExcelWriter(name)
demand_df.to_excel(writer, index = False)
writer.save()